In [19]:
using PyPlot

In [20]:
using Jacobi

In [21]:
remap(x,a,b) = ((1-x).*a + (1+x).*b)/2
remap(x,ab) = ((1-x).*ab[1] + (1+x).*ab[2])/2
dremap(x,a,b) = (b-a)/2
dremap(lim)   = (lim[2] - lim[1])/2

methods(dremap)

# 2 methods for generic function "dremap":
dremap(lim) at In[21]:4
dremap(x, a, b) at In[21]:3

In [22]:
function  ϕ_matrix(ψj,Q,M)
    ϕ = zeros(Q,M)
    ξ = zglj(Q)
    for i in 1:M
        for j in 1:Q
            ϕ[j,i] = ψj(i,ξ[j],M)
        end
    end
  return ϕ
end 
ϕ_matrix(ψ,10,2)

10x2 Array{Float64,2}:
 1.0       0.0     
 0.959767  0.040233
 0.869387  0.130613
 0.738962  0.261038
 0.582639  0.417361
 0.417361  0.582639
 0.261038  0.738962
 0.130613  0.869387
 0.040233  0.959767
 0.0       1.0     

In [23]:
function map_matrix(Nel)
    nos = zglj(Nel+1)
    matrix = zeros(Nel,2)
    matrix[1,1],matrix[1,2]=1,2
    for i in 2:Nel
        matrix[i,1] = i
        matrix[i,2] = i + 1
    end
    return round(Int64,matrix)
end

map_matrix (generic function with 1 method)

In [24]:
function ψ(p,E,M)
    if(p == 1)
        return  (1-E)/2
    elseif(p == 2)
        return  (1+E)/2
    end
end 

ψ (generic function with 1 method)

In [25]:
function cria_Ae(e,M,Q,e_M,jac)
    A = zeros(M,M)
    ϕ = ϕ_matrix(ψ,Q,M)
    for i in 1:M
        for j in 1:M
            temp = 0.0
            for q in 1:Q
                temp = temp + ϕ[q,i]*ϕ[q,j]*w[q]*jac
            end
            A[i,j] = temp
        end
    end
    return A
end

cria_Ae (generic function with 1 method)

In [26]:
function mass_matrix(map,Nl,Q,Ne,e_M,jac)
    z   = zglj(Q)
    w   = wglj(z,.0,.0)
    A   = zeros(Ne+1,Ne+1)
    for e in 1:Ne
        Ae= cria_Ae(e,Nl,Q,e_M,jac(e_M[e,:]))
        for i in 1:2
            for j in 1:2
                ig = map[e,i]
                jg = map[e,j]
                A[ig,jg] += Ae[i,j]
            end
        end
    end
    return A
end


mass_matrix (generic function with 1 method)

In [27]:
function elem_matrix(Nel,a,b,remap)
    nos = zglj(Nel+1)
    matrix = zeros(Nel,2)
    matrix[1,1],matrix[1,2]=remap(nos[1] ,a,b),remap(nos[2] ,a,b)
    for i in 2:Nel
        matrix[i,1] = remap(nos[i] ,a,b)
        matrix[i,2] = remap(nos[i+1] ,a,b)
    end
    return matrix
end

elem_matrix (generic function with 1 method)

In [28]:
remap2(x,a,b) = x
elem_matrix(3,-1,1,remap2)

3x2 Array{Float64,2}:
 -1.0       -0.447214
 -0.447214   0.447214
  0.447214   1.0     

In [29]:
function cria_Fe(f,e,Q,Ne,e_M::Array,ψ,jac)
    Fe = zeros(2)
    ξ  = zglj(Q)
    w  = wglj(ξ)
    for i in 1:2
        for q in 1:Q
            Fe[i] += jac(e_M[e,:]) * ψ(i,ξ[q],0) * f(remap(ξ[q],e_M[e,:]))*w[q]
        end
    end
    return Fe
end



cria_Fe (generic function with 1 method)

In [30]:
function fr_vector(fun,Nel,Q,e_M,ψ,dremap)
    fr = zeros(Nel+1)
    map = map_matrix(Nel)
    for e in 1:Nel
        Fe = cria_Fe(fun,e,Q,Nel,e_M,ψ,dremap)
        for i in 1:2
            ig = map[e,i]
            fr[ig] += Fe[i]
        end
    end
    return fr
end

fr_vector (generic function with 1 method)

In [45]:
Nel = 3
Q = 10
M = 2
a = -1
b = 1
#########################
fun(x) = sin(x*2*pi)
#########################
dremap2(e_M)= (e_M[1]-e_M[2])*0 +1
e_M = elem_matrix(Nel,a,b,remap)
z   = zglj(Nel+1)
w   = wglj(z,.0,.0)
map = map_matrix(Nel)
M   = mass_matrix(map,M,Q,Nel,e_M,dremap)
#fr = fr_vector(fun,Nel,Q,e_M,ψ,dremap)

#
#ue = fun(remap(zglj(Nel+1),a,b))'
#uh = M\fr
#ue - uh
M


LoadError: LoadError: BoundsError: attempt to access 4-element Array{Float64,1}:
 0.166667
 0.833333
 0.833333
 0.166667
  at index [5]
while loading In[45], in expression starting on line 14

In [32]:
jac = dremap(e_M[2,:])
#e_M

0.23991190372440974

In [33]:
plot(zglj(Nel+1),uh)
plot(zglj(Nel+1),ue')


LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.ValueError'>
ValueError('x and y must have same first dimension',)
  File "/usr/lib/pymodules/python2.7/matplotlib/pyplot.py", line 2987, in plot
    ret = ax.plot(*args, **kwargs)
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 4137, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 317, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 295, in _plot_args
    x, y = self._xy_from_xy(x, y)
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 237, in _xy_from_xy
    raise ValueError("x and y must have same first dimension")

while loading In[33], in expression starting on line 1

In [34]:
fun(remap(zglj(Nel+1),a,b))'

1x6 Array{Float64,2}:
 2.44929e-16  0.995529  -0.975598  0.975598  -0.995529  -2.44929e-16

In [35]:

plot(z,uh',label="aprox")
plot(z,fun(remap(zglj(Nel+1),a,b)),label="exact")
legend()

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.ValueError'>
ValueError('x and y must have same first dimension',)
  File "/usr/lib/pymodules/python2.7/matplotlib/pyplot.py", line 2987, in plot
    ret = ax.plot(*args, **kwargs)
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 4137, in plot
    for line in self._get_lines(*args, **kwargs):
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 317, in _grab_next_args
    for seg in self._plot_args(remaining, kwargs):
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 295, in _plot_args
    x, y = self._xy_from_xy(x, y)
  File "/usr/lib/pymodules/python2.7/matplotlib/axes.py", line 237, in _xy_from_xy
    raise ValueError("x and y must have same first dimension")

while loading In[35], in expression starting on line 2